In [ ]:
#Libraries

import tkinter as tk
import speech_recognition as sr
import requests
from bs4 import BeautifulSoup
from gtts import gTTS 
import os
import threading
import time
from googletrans import Translator

#Global varaiables

global text
global URL
global j

langs = {
    'auto': 'Automatic',
    'af': 'Afrikaans',
    'sq': 'Albanian',
    'am': 'Amharic',
    'ar': 'Arabic',
    'hy': 'Armenian',
    'az': 'Azerbaijani',
    'eu': 'Basque',
    'be': 'Belarusian',
    'bn': 'Bengali',
    'bs': 'Bosnian',
    'bg': 'Bulgarian',
    'ca': 'Catalan',
    'ceb': 'Cebuano',
    'ny': 'Chichewa',
    'zh-cn': 'Chinese Simplified',
    'zh-tw': 'Chinese Traditional',
    'co': 'Corsican',
    'hr': 'Croatian',
    'cs': 'Czech',
    'da': 'Danish',
    'nl': 'Dutch',
    'en': 'English',
    'eo': 'Esperanto',
    'et': 'Estonian',
    'tl': 'Filipino',
    'fi': 'Finnish',
    'fr': 'French',
    'fy': 'Frisian',
    'gl': 'Galician',
    'ka': 'Georgian',
    'de': 'German',
    'el': 'Greek',
    'gu': 'Gujarati',
    'ht': 'Haitian Creole',
    'ha': 'Hausa',
    'haw': 'Hawaiian',
    'iw': 'Hebrew',
    'hi': 'Hindi',
    'hmn': 'Hmong',
    'hu': 'Hungarian',
    'is': 'Icelandic',
    'ig': 'Igbo',
    'id': 'Indonesian',
    'ga': 'Irish',
    'it': 'Italian',
    'ja': 'Japanese',
    'jw': 'Javanese',
    'kn': 'Kannada',
    'kk': 'Kazakh',
    'km': 'Khmer',
    'ko': 'Korean',
    'ku': 'Kurdish (Kurmanji)',
    'ky': 'Kyrgyz',
    'lo': 'Lao',
    'la': 'Latin',
    'lv': 'Latvian',
    'lt': 'Lithuanian',
    'lb': 'Luxembourgish',
    'mk': 'Macedonian',
    'mg': 'Malagasy',
    'ms': 'Malay',
    'ml': 'Malayalam',
    'mt': 'Maltese',
    'mi': 'Maori',
    'mr': 'Marathi',
    'mn': 'Mongolian',
    'my': 'Myanmar (Burmese)',
    'ne': 'Nepali',
    'no': 'Norwegian',
    'ps': 'Pashto',
    'fa': 'Persian',
    'pl': 'Polish',
    'pt': 'Portuguese',
    'ma': 'Punjabi',
    'ro': 'Romanian',
    'ru': 'Russian',
    'sm': 'Samoan',
    'gd': 'Scots Gaelic',
    'sr': 'Serbian',
    'st': 'Sesotho',
    'sn': 'Shona',
    'sd': 'Sindhi',
    'si': 'Sinhala',
    'sk': 'Slovak',
    'sl': 'Slovenian',
    'so': 'Somali',
    'es': 'Spanish',
    'su': 'Sundanese',
    'sw': 'Swahili',
    'sv': 'Swedish',
    'tg': 'Tajik',
    'ta': 'Tamil',
    'te': 'Telugu',
    'th': 'Thai',
    'tr': 'Turkish',
    'uk': 'Ukrainian',
    'ur': 'Urdu',
    'uz': 'Uzbek',
    'vi': 'Vietnamese',
    'cy': 'Welsh',
    'xh': 'Xhosa',
    'yi': 'Yiddish',
    'yo': 'Yoruba',
    'zu': 'Zulu'
}
langs = dict(map(reversed, langs.items())) 

translator = Translator()

form_contents=[]
URL = input("Please enter the URL of the form: ")
lang_input = input("please enter the preferred language : ")
lang_input = lang_input.lower()
lang_input = lang_input.capitalize()
language = langs[lang_input]

#Function for taking speech input

def speech_input():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        #print("Please speak:")
        print(translator.translate("Please speak :", dest=language).text)
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            text = text.capitalize()
            text1 = translator.translate(text).text
            print(translator.translate("You said : {}".format(text1), dest=language).text)
            form_contents.append(text1)
            return text1
            #form_contents.append(text1)
            #print("You said : {}".format(text))          
        except:
            no_input = translator.translate("Please speak again",dest=language).text
            print(no_input)
            repeat = gTTS(text = no_input, lang = language, slow = False)
            repeat.save("text"+".mp3")
            os.system("text"+".mp3")
            speech_input()


                
#Function for getting HTML Form fields               
            
def text_to_speech(j):
        text = "Please enter the " + form_fields[j] 
        text1 = translator.translate(text, dest=language).text
        print(text1)
        speech = gTTS(text = text1, lang = language, slow = False)
        speech.save("text"+str(j)+".mp3")
        os.system("start text"+str(j)+".mp3")
        j=j+1           

headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}        
page = requests.get(URL, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
input =soup.find_all("input")
div = soup.find_all('div',class_='label-flex')
label = soup.find_all("label")
print("-------------------------------")
form_fields = []
try:
    for tab in input:
        if tab["type"] == 'text' or tab["type"] == 'password':
        #if i["type"] != "hidden" and i["name"] != "website":
            for j in label:
                if tab["id"] == j["for"]:
                    form_fields.append(j.text.capitalize())
                    break
except:
    pass

try:
    for tab in div:
        text = tab.label.text
        form_fields.append(text)
except:
    pass

print(form_fields)
j=0

for i in range(len(form_fields)):
        #time.sleep(2)
        threading.Thread(target=text_to_speech(j)).start()
        time.sleep(1)
        threading.Thread(target=speech_input()).start()
        j=j+1


print(form_contents)
time.sleep(1)
master=tk.Tk(className = 'Form Details')
master.geometry('400x600')

for i in range(0,len(form_contents)):
    text=tk.Label(master,text=form_fields[i]).grid(row=i)
    e1=tk.Entry(master)
    e1.insert(30,form_contents[i])
    e1.grid(row=i,column=1)
master.mainloop()
